In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import time


load_dotenv()

True

api key 설정, 헬퍼 함수 정의

In [5]:
api_key = os.environ.get("OpenAI_API_KEY")

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

Assistant 생성

In [ ]:
client = OpenAI(api_key=api_key)
assistant = client.beta.assistants.create(
    name="Recipe Bot",
    instructions="""
        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:

        Selected ingredients: A list of ingredients chosen by the user.
        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).

        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:

        - A title describing the dish.
        - A brief summary of the dish.
        - A list of ingredients (including the selected ones).
        - Step-by-step cooking instructions.

        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.

        If the selected ingredients are insufficient for the recipe, suggest additional ingredients and explain why they are necessary.
        
        Your responses should be in Korean.
    """,
    model="gpt-4o-mini"
)

show_json(assistant)
ASSISTANT_ID = assistant.id

{'id': 'asst_ICzPekzwrJOLSvZ7uJO5nKEi',
 'created_at': 1731983405,
 'description': None,
 'instructions': "\n        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:\n\n        Selected ingredients: A list of ingredients chosen by the user.\n        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).\n\n        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:\n\n        - A title describing the dish.\n        - A brief summary of the dish.\n        - A list of ingredients (including the selected ones).\n        - Step-by-step cooking instructions.\n\n        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.\n\n        If the selected ingredients are insuffic

Thread 생성
- 스레드에 메시지 추가
- 스레드 실행(run)
- 스레드 상태(대기, 작업, 완료 등) 확인
- 스레드 최신 메시지 조회

In [7]:
def create_thread():
    thread = client.beta.threads.create()
    return thread

thread = create_thread()
show_json(thread)
THREAD_ID = thread.id

{'id': 'thread_rxynZP6z5oUekIyFFFWHVFij',
 'created_at': 1731983405,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [ ]:
def wait_on_run(run, thread_id):
    # run이 완료될 때까지 대기
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )
        time.sleep(0.5)  # 서버 부하를 줄이기 위한 API 요청 사이 대기 시간
    return run

def submit_message(assistant_id, thread_id, user_input):
    ingredients, cooking_style = user_input[0], user_input[1]
    
    message_content = f"""
    Here are the user's inputs:
    - Selected ingredients: {ingredients}
    - Preferred cooking style: {cooking_style}
    
    Please generate three recipes considering the above inputs.
    """
    
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_content
    )

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    return run

def get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")

def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")
    print("-" * 50)

def ask(assistant_id, thread_id, user_input):
    run = submit_message(assistant_id, thread_id, user_input)
    run = wait_on_run(run, thread_id)
    response_data = get_response(thread_id).data
    if response_data:
        print_message(response_data[-1])  # -2 : [USER]까지
    else:
        print("응답이 없습니다.")
    
    return run

In [11]:
user_input = ["양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장", "약간 맵게 해줘"]
run = ask(ASSISTANT_ID, THREAD_ID, user_input)

[USER]

    Here are the user's inputs:
    - Selected ingredients: 양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장
    - Preferred cooking style: 약간 맵게 해줘
    
    Please generate three recipes considering the above inputs.
    

[ASSISTANT]
### 레시피 1: 매운 김치 삼겹살 파스타

**요약:** 김치의 매콤하고 감칠맛 나은 풍미와 삼겹살의 고소한 맛이 어우러진 창의적인 파스타 요리입니다. 이국적인 맛을 좋아하는 분께 제격입니다.

**재료:**
- 파스타 면 200g
- 삼겹살 150g (한 입 크기로 자른다)
- 양파 1개 (슬라이스)
- 감자 1개 (작게 큐브)
- 양송이 100g (슬라이스)
- 대파 1대 (송송 썰기)
- 김치 1컵
- 간장 2큰술
- 고춧가루 1큰술 (매운 맛을 위해)
- 식용유

**조리 방법:**
1. 큰 냄비에 물을 끓이고 소금을 넣은 후, 파스타 면을 패키지 지시에 따라 삶습니다. 삶은 면은 체에 걸러 물기를 빼고 둡니다.
2. 팬에 식용유를 두르고 중불에서 삼겹살을 바삭하게 익힙니다.
3. 삼겹살이 익어가면 양파와 감자를 추가하고 함께 볶아 부드러워질 때까지 조리합니다.
4. 양송이를 넣고 조금 더 볶은 후, 김치와 간장을 추가하고 잘 섞습니다.
5. 고춧가루를 넣고 전체 재료가 잘 어우러지도록 볶아줍니다.
6. 삶은 파스타 면을 넣고 모든 재료가 잘 혼합될 때까지 볶은 후, 접시에 담아 대파를 뿌리고 즐기세요!

---

### 레시피 2: 매운 양파와 감자 볶음

**요약:** 양파와 감자의 부드러움, 그리고 김치의 매콤함이 조화롭게 어우러진 한국식 볶음 요리입니다. 간단하게 밥과 함께 먹기 좋은 반찬입니다.

**재료:**
- 감자 2개 (작은 큐브)
- 양파 1개 (슬라이스)
- 대파 1대 (송송 썰기)
- 김치 1컵
- 간장 1큰술
- 고